In [70]:
spark

res18: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@52256b8c


In [71]:
import scala.collection.mutable.WrappedArray
import org.apache.spark.sql.{Column, Row}
import org.apache.spark.sql.functions.{col,when}

import scala.collection.mutable.WrappedArray
import org.apache.spark.sql.{Column, Row}
import org.apache.spark.sql.functions.{col, when}


In [72]:
val validations = "/data/validation.json"
val data = spark.read.option("header", true).csv("/data/data.csv")

validations: String = /data/validation.json
data: org.apache.spark.sql.DataFrame = [id: string, age: string]


In [73]:
data.printSchema()
data.show


root
 |-- id: string (nullable = true)
 |-- age: string (nullable = true)

+---+----+
| id| age|
+---+----+
|  1|  10|
|  2|  18|
|  3| 100|
|  4| 120|
|  5|null|
|  6|text|
+---+----+



In [74]:
def validateRange(name: Column, max: Integer, min: Integer=null): Column = {
    println(s"Validator: validateRange, column: ${name}, params: ${min}, ${max}")
    if(null == max) {
        name >=min    
    }else{
        name >= min && name < max
    }
}

validateRange: (name: org.apache.spark.sql.Column, max: Integer, min: Integer)org.apache.spark.sql.Column


In [75]:
def isNaN(name: Column): Column = {
    println(s"Validator: isNaN, column: ${name}")
    name.isNaN
}

isNaN: (name: org.apache.spark.sql.Column)org.apache.spark.sql.Column


In [76]:
def selectValidator(validator: Row): Column = {
    print(validator)
    val column = col(validator.getString(0))
    val func = validator.getString(2)
    println(s"column:$column")
    println(s"func:$func")
    func match {
        case "isNaN" => isNaN(column)
        case "range" => validateRange(column, validator.getLong(3).toInt, validator.getLong(4).toInt) 
    }
}

selectValidator: (validator: org.apache.spark.sql.Row)org.apache.spark.sql.Column


In [77]:
import org.apache.spark.sql.DataFrame   
def transformValidations(dfValidations: DataFrame )={
       val dfExplode=  dfValidations.withColumn("validators",explode(col("validators")))
      .withColumn("level",$"validators.level")
      //.withColumn("validator",$"validators.validator")
      .withColumn("func",$"validators.validator.func")
      .withColumn("max",$"validators.validator.max")
      .withColumn("min",$"validators.validator.min")
    val columnasQuery = Seq("column","level","func","max","min")
    val returndf = dfExplode.select(columnasQuery.head,columnasQuery.tail:_*)
       returndf
   }

import org.apache.spark.sql.DataFrame
transformValidations: (dfValidations: org.apache.spark.sql.DataFrame)org.apache.spark.sql.DataFrame


In [78]:
def crearWiths(row: Row): List[Column] = {
    val column = row.getAs[String]("column")
    
    val r = row.getAs[WrappedArray[Row]]("validators")
    println(s"${column} -> ${r}")
    val cols = List[Column]()
    for(ers <- r) {
        val lvl = ers.getAs[String]("level")
        val validator = ers.getAs[Row]("validator")
        cols :+ selectValidator(col(column), validator)
    }
    cols
}

<console>: 50: error: too many arguments (2) for method selectValidator: (validator: org.apache.spark.sql.Row)org.apache.spark.sql.Column

In [79]:
def crearWithsOr (row: Row) = {
    println(row)
    lit(1)
}

crearWithsOr: (row: org.apache.spark.sql.Row)org.apache.spark.sql.Column


In [80]:
spark.read.option("multiline","true").json(validations).show()

+------+--------------------+
|column|          validators|
+------+--------------------+
|   age|[{error, {isNaN, ...|
+------+--------------------+



In [81]:
def getErrors() = {
    val df = spark.read.option("multiline","true").json(validations)
    df.show()
    val newValidations = transformValidations(df)
    newValidations.show()
    newValidations.collect().map(selectValidator(_)).reduce(_ or _)
    //.show
    //df.
    //val creates = df.collect().map(crearWiths(_))
    //creates
}

getErrors: ()org.apache.spark.sql.Column


In [82]:
getErrors()

+------+--------------------+
|column|          validators|
+------+--------------------+
|   age|[{error, {isNaN, ...|
+------+--------------------+

+------+-----+-----+----+----+
|column|level| func| max| min|
+------+-----+-----+----+----+
|   age|error|isNaN|null|null|
|   age| warn|range| 120|  18|
+------+-----+-----+----+----+

[age,error,isNaN,null,null]column:age
func:isNaN
Validator: isNaN, column: age
[age,warn,range,120,18]column:age
func:range
Validator: validateRange, column: age, params: 18, 120


res21: org.apache.spark.sql.Column = (isnan(age) OR ((age >= 18) AND (age < 120)))


In [83]:
data.withColumn("error", getErrors).show

+------+--------------------+
|column|          validators|
+------+--------------------+
|   age|[{error, {isNaN, ...|
+------+--------------------+

+------+-----+-----+----+----+
|column|level| func| max| min|
+------+-----+-----+----+----+
|   age|error|isNaN|null|null|
|   age| warn|range| 120|  18|
+------+-----+-----+----+----+

[age,error,isNaN,null,null]column:age
func:isNaN
Validator: isNaN, column: age
[age,warn,range,120,18]column:age
func:range
Validator: validateRange, column: age, params: 18, 120
+---+----+-----+
| id| age|error|
+---+----+-----+
|  1|  10|false|
|  2|  18| true|
|  3| 100| true|
|  4| 120|false|
|  5|null| null|
|  6|text| null|
+---+----+-----+

